In [3]:
!pip install transformers sentencepiece datasets faiss-cpu langchain openai tiktoken spacy nltk
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Importing and Loading Pretrained Transformer model and it's corresponding tokenizer from the transformer library by Hugging face

T0pp (T0 Prime Plus) is an open-source model trained by BigScience and is designed for zero-shot and few-shot text generation tasks, meaning it can answer prompts without additional fine-tuning.
It is based on the T5 (Text-to-Text Transfer Transformer) architecture, meaning it expects text input and produces text output.

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "bigscience/T0pp"  # Open-source and available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/44.5G [00:00<?, ?B/s]

#Further Preprocessing
This script below  is to preprocess legal text for further analysis, such as text classification, NLP tasks, or legal document processing. It uses the spaCy library for linguistic processing and re (Regular Expressions) for text cleaning.

This script is designed to extract named entities from a given text using spaCy's Named Entity Recognition (NER). It identifies important entities such as organizations, dates, and legal terms from the text.

Processing Steps:
Converts to lowercase.
Removes punctuation.
Tokenizes text.
Removes stopwords.
Lemmatizes words.

In [5]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

def preprocess_legal_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]  # Lemmatization
    return " ".join(tokens)

sample_text = "The contract is valid for 10 years, terminating on December 31, 2030."
print(preprocess_legal_text(sample_text))

contract valid 10 year terminate december 31 2030


# Extracting named entities from the given text using spaCy's Named Entity Recognition (NER)

This script is designed to extract named entities from a given text using spaCy's Named Entity Recognition (NER). It identifies important entities such as organizations, dates, and legal terms from the text.

Extracts important legal entities (e.g., company names, contracts, dates).
✅ Automates legal text analysis (e.g., contract processing, case law summaries).
✅ Can be expanded to detect laws (LAW label) with a more advanced legal NLP model.

In [6]:
import spacy

def extract_legal_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

sample_text = "Apple Inc. signed a contract with Microsoft on June 15, 2023."
print(extract_legal_entities(sample_text))

[('Apple Inc.', 'ORG'), ('Microsoft', 'ORG'), ('June 15, 2023', 'DATE')]


In [7]:
import torch

# Embeding text using a transformer model
This script embeds legal text using a transformer model, indexes the embeddings with FAISS, and retrieves the most relevant legal document for a given query. Let's break it down step by step.

Tokenizes input text and converts it into tensors for the model.
Uses with torch.no_grad() to disable gradient computation (speeds up inference).
Extracts the last_hidden_state from the model’s output.
Averages the token embeddings (mean(dim=1)) to get a single vector representation for the entire text.
Converts the tensor to a NumPy array for compatibility with FAISS.

# Summary: What Does This Script Do?
Loads a pretrained transformer model (all-MiniLM-L6-v2) for sentence embeddings.
Converts legal documents into embeddings (dense vector representations).
Stores the embeddings in a FAISS index for fast similarity search.
Embeds a query and retrieves the most relevant legal document using nearest neighbor search.


In [8]:
import faiss
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

embedding_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embedding_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def embed_text(text):
    inputs = embedding_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = embedding_model(**inputs)
    embedding = output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Ensure 1D vector
    return embedding

legal_docs = [
    "A contract is legally binding if signed by both parties.",
    "An NDA prevents disclosure of confidential information.",
    "A non-compete agreement prohibits working for a competitor."
]

doc_embeddings = np.array([embed_text(doc) for doc in legal_docs])

print("Embeddings Shape:", doc_embeddings.shape)  # Should be (num_samples, embedding_dim)

index = faiss.IndexFlatL2(doc_embeddings.shape[1])  # Dimension should match embedding size
index.add(doc_embeddings)

query = "What happens if I break an NDA?"
query_embedding = embed_text(query).reshape(1, -1)  # Reshape for FAISS
_, retrieved_indices = index.search(query_embedding, 1)

print(f"Best matching legal text: {legal_docs[retrieved_indices[0][0]]}")

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Embeddings Shape: (3, 384)
Best matching legal text: An NDA prevents disclosure of confidential information.


# Checks whether a GPU (CUDA) is available for PyTorch to use.

In [9]:
import torch
print("GPU Available:", torch.cuda.is_available())

GPU Available: True


# Takes a legal query as input (e.g., "What happens if I break an NDA?").
Encodes the query into token IDs using tokenizer(), preparing it for the model.
Generates a response using model.generate() with a max length of 100 tokens.
Decodes the model's output back into readable text using tokenizer.decode()

In [10]:
def legal_chatbot(query):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

query = "What happens if I break an NDA?"
print(legal_chatbot(query))

I will be sued
